In [35]:
import torch
import torchvision

class TrainData:
    def __init__(self):
        self._download_train_data()
        
    def _download_train_data(self):
        orig_train = torchvision.datasets.CIFAR10('data', train=True, transform=torchvision.transforms.ToTensor(), download=True)

        classes = tuple(orig_train.classes)

        keep_labels = (classes.index('cat'), classes.index('dog'))
        aux_labels = tuple(classes.index(a) for a in ['airplane', 'automobile', 'bird', 'ship', 'truck'])
        n = 10000
        auxn = 25000
        channels = 3
        w = 32
        h = 32
        X = torch.zeros((n, channels, w, h))
        y = torch.zeros((n,), dtype=torch.long)
        auxX = torch.zeros((auxn, channels, w, h))

        j = 0
        auxj = 0
        for x,label in orig_train:
            if label in keep_labels:
                X[j,:,:,:] = x
                y[j] = keep_labels.index(label)
                j += 1
            if label in aux_labels:
                auxX[auxj,:,:,:] = x
                auxj += 1
        if j != n:
            raise Exception("Wrong number of valid examples")
        if auxj != auxn:
            raise Exception("Wrong number of aux examples")
        self._X = X
        self._y = y
        self._n = n
        self._auxX = auxX
        self._auxn = auxn
        
    def in_distribution_dataset(self):
        return [(self._X[i], self._y[i]) for i in range(self._n)]
        
    def mixed_dataset(self):
        ind = [(self._X[i], (self._y[i], 0)) for i in range(self._n)]
        ood = [(self._auxX[i], (0.5, 1)) for i in range(self._auxn)]
        return ind + ood
        
train_data = TrainData()
print("Have training data")

Files already downloaded and verified
Have training data


In [52]:

class TestData:
    def __init__(self):
        self._download_test_data()
        
    def _download_test_data(self):
        orig_train = torchvision.datasets.CIFAR10('data', train=False, transform=torchvision.transforms.ToTensor(), download=True)

        classes = tuple(orig_train.classes)

        keep_labels = (classes.index('cat'), classes.index('dog'))
        ood_labels = tuple(classes.index(a) for a in ['deer','frog','horse'])
        n = 2000
        oodn = 3000
        channels = 3
        w = 32
        h = 32
        X = torch.zeros((n, channels, w, h))
        y = torch.zeros((n,), dtype=torch.long)
        oodX = torch.zeros((oodn, channels, w, h))

        j = 0
        oodj = 0
        for x,label in orig_train:
            if label in keep_labels:
                X[j,:,:,:] = x
                y[j] = keep_labels.index(label)
                j += 1
            if label in ood_labels:
                oodX[oodj,:,:,:] = x
                oodj += 1
        if j != n:
            raise Exception(f"Wrong number of valid examples {j} {n}")
        if oodj != oodn:
            raise Exception(f"Wrong number of ood examples {oodj} {oodn}")
        self._X = X
        self._y = y
        self._n = n
        self._oodX = oodX
        self._oodn = oodn
        
    def in_distribution_dataset(self):
        return [(self._X[i], self._y[i]) for i in range(self._n)]
        
#    def mixed_dataset(self):
#        ind = [(self._X[i], (self._y[i], 0)) for i in range(self._n)]
#        ood = [(self._auxX[i], (0.5, 1)) for i in range(self._auxn)]
#        return ind + ood
        
test_data = TestData()
print("Have test data")

Files already downloaded and verified
Have test data


In [238]:
class SimpleCnnModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Conv2d(3, 6, 5),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, 2),
            torch.nn.Conv2d(6, 16, 5),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, 2),
            torch.nn.Flatten(),
            torch.nn.Linear(16 * 5 * 5, 40),
            #torch.nn.ReLU(),
            #torch.nn.Linear(40, 40),
            torch.nn.ReLU(),
            torch.nn.Linear(40, 2),
        )

    def forward(self, x):
        return self.layers(x)


In [243]:
class Experiment:
    def __init__(self):
        global train_data
        global test_data
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.model = SimpleCnnModel().to(self.device)
        self.dataloader = torch.utils.data.DataLoader(
            train_data.in_distribution_dataset(),
            batch_size=128,
            shuffle=True
        )
        self.test_dataloader = torch.utils.data.DataLoader(
            test_data.in_distribution_dataset(),
            batch_size=64,
            shuffle=False
        )
        self.loss_fn = torch.nn.CrossEntropyLoss(reduction='sum')
        self.optimizer = torch.optim.Adam(self.model.parameters(), weight_decay=1.6, lr=0.001)
        self.num_epochs = 200
        
    def _train(self):
        print(self.device)
        for epoch in range(self.num_epochs):
            #print(f"=== Epoch {epoch}===")
            running_loss = torch.zeros(())
            train_accuracy = torch.zeros(())
            running_count = 0
            for inputs, labels in self.dataloader:
                self.optimizer.zero_grad()
                outputs = self.model(inputs.to(self.device))
                predictions = outputs.detach().argmax(dim=1)
                loss = self.loss_fn(outputs, labels.to(self.device))
                loss.backward()
                self.optimizer.step()
                running_loss += loss.detach().to('cpu')
                train_accuracy += (predictions.to('cpu') == labels).sum()
                running_count += outputs.shape[0]
                    
            #print('    Loss', running_loss.item() / running_count)  

            if epoch % 5 == 4:
                test_accuracy = torch.zeros(())
                test_count = 0
                with torch.no_grad():
                    for inputs, labels in self.test_dataloader:
                        outputs = self.model(inputs.to(self.device))
                        predictions = outputs.argmax(dim=1)
                        test_accuracy += (predictions.to('cpu') == labels).sum()
                        test_count += outputs.shape[0]
                print('Epoch', f'{epoch:2}',
                      'Loss', f'{running_loss.item() / running_count:20}',
                      'Train', f'{train_accuracy.item() / running_count:10}',
                      'Test', test_accuracy.item() / test_count)

In [ ]:
Experiment()._train()

cuda:0
Epoch  4 Loss     0.65898291015625 Train     0.6049 Test 0.605
Epoch  9 Loss    0.643774755859375 Train     0.6281 Test 0.6295
Epoch 14 Loss     0.62853681640625 Train     0.6503 Test 0.6325
Epoch 19 Loss     0.61167587890625 Train      0.668 Test 0.645
Epoch 24 Loss    0.597730615234375 Train     0.6764 Test 0.6495
Epoch 29 Loss      0.5872794921875 Train      0.687 Test 0.673
Epoch 34 Loss    0.573045654296875 Train      0.701 Test 0.6675
Epoch 39 Loss     0.56748330078125 Train     0.7032 Test 0.6935
Epoch 44 Loss     0.55989052734375 Train     0.7089 Test 0.6885
Epoch 49 Loss    0.554943310546875 Train     0.7127 Test 0.6935
Epoch 54 Loss      0.5503865234375 Train     0.7176 Test 0.6955
Epoch 59 Loss     0.54890595703125 Train     0.7182 Test 0.691
Epoch 64 Loss      0.5453076171875 Train     0.7185 Test 0.7055
Epoch 69 Loss       0.539388671875 Train     0.7244 Test 0.7025
Epoch 74 Loss     0.53211455078125 Train     0.7316 Test 0.709
Epoch 79 Loss    0.530205224609375 Tra